In [1]:
import math

In [27]:
board = [[1, 0, 1, 0],
          [1, 0, 1, 0],
          [1, 1, 4, 1],
         [1, 1, 1, 1]]

In [28]:
pp = (3, 0)
np = (2, 3)

dx = np[1] - pp[1]
dy = np[0] - pp[0]


def tr(v):
    f, w = math.modf(v)
    if f >= 0.5:
        return math.ceil(v)
    else:
        return math.floor(v)

if abs(dx) > abs(dy):
    if pp[1] > np[1]:
        pp, np = np, pp 
    
    
    a = (np[0] - pp[0]) / (np[1] - pp[1])
    y = pp[0]
    for x in range(pp[1], np[1] + 1):
        if board[tr(y)][x] == 4:
            print('have wall')
        y += a

else:
    if pp[0] > np[0]:
        pp, np = np, pp
        
    a = (np[1] - pp[1]) / (np[0] - pp[0])
    x = pp[1]
    
    for y in range(pp[0], np[0] + 1):
        if board[y][tr(x)] == 4:
            print('have wall')
        x += a




have wall


1
4
